# [TUTORIAL] Constrained Task Planning (Sweep)

This tutorial is written to instruct basic usage of the task & motion planning pipeline  
One Indy7 robot and several environment geometries will be added and floor-wiping task will be conducted.  
Here, the wiping task is defined as 1) contact with floor, 2) starting waypoint, 3) goal waypoint.  
Thus, any motion that satisfies the constraint will be generated; it may not look like real wiping.  

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,0,0), (0,0,0)),
                INDY_IP)]
              , connection_list=[False])

connection_list
[False]


## create scene builder

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


## init planning pipeline

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [5]:
from pkg.geometry.geometry import *

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (0.3,0.5,0.01), (0.5,0,-0.005), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
floor_col = gscene.create_safe(GEOTYPE.BOX, "floor_col", "base_link", (0.3,0.5,0.01), (0.5,0,-0.006), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=False, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,-0.005), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=True, collision=False)
goal_col = gscene.create_safe(GEOTYPE.BOX, "goal_col", "base_link", (0.1,0.1,0.01), (0.3,-0.4,-0.006), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=False, fixed=True, collision=True)

 * Environment: production
Please create a subscriber to the marker
   Use a production WSGI server instead.
 * Debug mode: off


In [6]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add wp

In [7]:
wp11 = gscene.create_safe(GEOTYPE.BOX, "wp11", "base_link", (0.08,0.08,0.01), (0.6,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp12 = gscene.create_safe(GEOTYPE.BOX, "wp12", "base_link", (0.08,0.08,0.01), (0.6,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line1 = gscene.create_safe(GEOTYPE.BOX, "wline1", "base_link", (0.01,0.5,1e-6), (0.6,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

wp21 = gscene.create_safe(GEOTYPE.BOX, "wp21", "base_link", (0.08,0.08,0.01), (0.5,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp22 = gscene.create_safe(GEOTYPE.BOX, "wp22", "base_link", (0.08,0.08,0.01), (0.5,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line2 = gscene.create_safe(GEOTYPE.BOX, "wline2", "base_link", (0.01,0.5,1e-6), (0.5,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

wp31 = gscene.create_safe(GEOTYPE.BOX, "wp31", "base_link", (0.08,0.08,0.01), (0.4,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp32 = gscene.create_safe(GEOTYPE.BOX, "wp32", "base_link", (0.08,0.08,0.01), (0.4,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line3 = gscene.create_safe(GEOTYPE.BOX, "wline3", "base_link", (0.01,0.5,1e-6), (0.4,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

## add brush

In [8]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="base_link", dims=(0.1,0.07,0.02), 
                   center=(0.3,-0.4,0.04), rpy=(0,0,0), color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="base_link", dims=(0.1,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="base_link", dims=(0.1,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")

## add box

In [9]:
gbox1 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box1", link_name="base_link", dims=(0.05,0.05,0.05), 
                   center=(0.5,0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

# gbox2 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box2", link_name="base_link", dims=(0.05,0.05,0.05), 
#                    center=(0.4,-0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [11]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [12]:
pscene.create_binder(bname="grip0", gname="grip0", rname="indy0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepTool, point=(0,0,-0.015), rpy=(0,0,0))

## add objects

In [13]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask, BoxObject

In [14]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush_face = gscene.NAME_DICT["brush_face"]
brush = pscene.create_object(oname="brush", gname="brush_body", _type=CustomObject, 
                             action_points_dict = {"handle": Grasp2Point("handle", brush_handle, [0,0,0], [np.pi/2,0,0]),
                                                   "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

In [15]:
box1 = pscene.create_object(oname="box1", gname="box1", _type=BoxObject, hexahedral=True)
# box2 = pscene.create_object(oname="box2", gname="box2", _type=BoxObject, hexahedral=True)

In [16]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constarint.constraint_subject import SweepLineTask

ImportError: No module named constarint.constraint_subject

In [17]:
sweep1 = pscene.create_object(oname="sweep1", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp11": SweepPoint("wp11", wp11, [0,0,0.005], [0,0,0]),
                                                   "wp12": SweepPoint("wp12", wp12, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line1)
sweep2 = pscene.create_object(oname="sweep2", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp21": SweepPoint("wp21", wp21, [0,0,0.005], [0,0,0]),
                                                   "wp22": SweepPoint("wp22", wp22, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line2)
sweep3 = pscene.create_object(oname="sweep3", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp31": SweepPoint("wp31", wp31, [0,0,0.005], [0,0,0]),
                                                   "wp32": SweepPoint("wp32", wp32, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line3)

NameError: name 'SweepLineTask' is not defined

In [18]:
# sweep = pscene.create_object(oname="sweep", gname="floor", _type=SweepTask, 
#                              action_points_dict = {"wp11": SweepPoint("wp11", wp11, [0,0,0.005], [0,0,0]),
#                                                    "wp12": SweepPoint("wp12", wp12, [0,0,0.005], [0,0,0]),
#                                                    "wp13": SweepPoint("wp13", wp13, [0,0,0.005], [0,0,0]),
#                                                    "wp14": SweepPoint("wp14", wp14, [0,0,0.005], [0,0,0]),
#                                                    "wp15": SweepPoint("wp15", wp15, [0,0,0.005], [0,0,0]),
#                                                    "wp21": SweepPoint("wp21", wp21, [0,0,0.005], [0,0,0]),
#                                                    "wp22": SweepPoint("wp22", wp22, [0,0,0.005], [0,0,0]),
#                                                    "wp23": SweepPoint("wp23", wp23, [0,0,0.005], [0,0,0]),
#                                                    "wp24": SweepPoint("wp24", wp24, [0,0,0.005], [0,0,0]),
#                                                    "wp25": SweepPoint("wp25", wp25, [0,0,0.005], [0,0,0]),
#                                                    "wp31": SweepPoint("wp31", wp31, [0,0,0.005], [0,0,0]),
#                                                    "wp32": SweepPoint("wp32", wp32, [0,0,0.005], [0,0,0]),
#                                                    "wp33": SweepPoint("wp33", wp33, [0,0,0.005], [0,0,0]),
#                                                    "wp34": SweepPoint("wp34", wp34, [0,0,0.005], [0,0,0]),
#                                                    "wp35": SweepPoint("wp35", wp35, [0,0,0.005], [0,0,0])})

### planners

In [19]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

## motion filters

In [20]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
TOOL_LINK = "indy0_tcp"
TOOL_LINK_BUNDLE = ["indy0_tcp", "indy0_link6"]

gcheck = GraspChecker(pscene, end_link_couple_dict= {TOOL_LINK: TOOL_LINK_BUNDLE, "base_link":["base_link"]})
rcheck = ReachChecker(pscene)
checkers_all = [gcheck, rcheck]
# lcheck = LatticedChecker(pscene, end_link_couple_dict= {TOOL_LINK: TOOL_LINK_BUNDLE, "base_link":["base_link"]})
# checkers_all.append(lcheck)

In [21]:
mplan.motion_filters = checkers_all

## Test plan

In [22]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.update_state(crob.home_pose)

In [28]:
gtimer.reset()
# goal_nodes = [("goal",)]
# goal_nodes = [("goal", "floor", "floor", 2, 2, 2)]
goal_nodes = [("floor", "goal", 2, 2, 2)]
gtimer.tic("plan")
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=300, multiprocess=True, timeout=1, timeout_constrained=1)
gtimer.toc("plan")
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

Use 20/20 agents
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 1) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 1, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 1, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 1, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 0, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 1, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 0, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 1, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 1) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 1) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 1) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0) = fail
nod

node: ('grip0', 'floor', 0, 0, 0)->('goal', 'floor', 0, 0, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 0, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 1) = fail
node: ('grip0', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 0, 1) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 1) = fail
node: ('floor', 'grip0', 1, 0, 0)->('floor', 'grip0', 2, 0, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0) = fail
node: ('floor', 'grip0', 0, 0, 0)->('floor', 'grip0', 1, 0, 0) = success
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 1, 0) = fail
branching: 2->12 (0.55/300.0 s, steps/err: 37(48.730134964 ms)/0.00143337093396)
node: ('floor', 'grip0', 0, 0, 0)->('floor', 'grip0', 1, 0, 0) = success
node: ('grip0', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 0, 1) = fail
branching: 1->13 (0.58/300.0 s, steps/err: 36(388.157129288 ms)/0.00146447839418)
node: ('grip0', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0)

branching: 2->29 (1.16/300.0 s, steps/err: 31(57.804107666 ms)/0.00205545638532)
node: ('floor', 'goal', 0, 0, 0)->('floor', 'grip0', 0, 0, 0) = fail
node: ('floor', 'grip0', 0, 0, 0)->('floor', 'grip0', 0, 1, 0) = fail
node: ('grip0', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 1, 0) = fail
node: ('floor', 'grip0', 0, 0, 0)->('floor', 'grip0', 0, 1, 0) = success
branching: 20->30 (1.23/300.0 s, steps/err: 31(241.48106575 ms)/0.00169424176032)
node: ('floor', 'grip0', 0, 0, 0)->('floor', 'grip0', 1, 0, 0) = success
node: ('floor', 'grip0', 0, 0, 0)->('floor', 'grip0', 0, 1, 0) = success
branching: 2->32 (1.28/300.0 s, steps/err: 34(255.312919617 ms)/0.00196483401859)
branching: 3->31 (1.28/300.0 s, steps/err: 11(31.6128730774 ms)/0.000616633184196)
node: ('floor', 'grip0', 0, 0, 1)->('floor', 'grip0', 0, 0, 2) = fail
node: ('floor', 'goal', 0, 0, 0)->('grip0', 'goal', 0, 0, 0) = fail
node: ('floor', 'grip0', 0, 0, 0)->('floor', 'grip0', 1, 0, 0) = success
node: ('grip0', 'floor', 0, 0, 0)

node: ('floor', 'grip0', 0, 0, 0)->('floor', 'goal', 0, 0, 0) = success
node: ('floor', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 0, 0) = success
branching: 1->45 (1.76/300.0 s, steps/err: 44(158.968925476 ms)/0.00174206087691)
node: ('floor', 'grip0', 1, 0, 0)->('floor', 'grip0', 2, 0, 0) = fail
node: ('floor', 'grip0', 0, 0, 1)->('floor', 'grip0', 0, 0, 2) = fail
node: ('floor', 'grip0', 0, 0, 0)->('floor', 'grip0', 0, 1, 0) = success
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 1, 0) = fail
node: ('floor', 'grip0', 1, 0, 0)->('floor', 'grip0', 2, 0, 0) = fail
node: ('floor', 'goal', 0, 0, 0)->('floor', 'goal', 0, 1, 0) = fail
node: ('floor', 'grip0', 0, 0, 0)->('floor', 'grip0', 1, 0, 0) = success
branching: 20->46 (1.77/300.0 s, steps/err: 24(30.944108963 ms)/0.00164082609702)
node: ('grip0', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 1, 0) = fail
branching: 0->44 (1.78/300.0 s, steps/err: 36(323.482036591 ms)/0.00161286974546)
node: ('grip0', 'floor', 0, 0, 0)->('grip0', 

node: ('floor', 'floor', 0, 0, 0)->('floor', 'grip0', 0, 0, 0) = success
node: ('floor', 'grip0', 0, 0, 0)->('floor', 'grip0', 1, 0, 0) = success
branching: 21->60 (2.68/300.0 s, steps/err: 48(288.118124008 ms)/0.00207761557957)
branching: 57->59 (2.68/300.0 s, steps/err: 14(37.5940799713 ms)/0.00190571706656)
node: ('floor', 'grip0', 0, 1, 0)->('floor', 'grip0', 0, 2, 0) = fail
node: ('grip0', 'floor', 0, 0, 0)->('goal', 'floor', 0, 0, 0) = fail
node: ('floor', 'goal', 0, 0, 0)->('floor', 'goal', 1, 0, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0) = fail
node: ('floor', 'grip0', 1, 0, 0)->('floor', 'grip0', 2, 0, 0) = fail
node: ('floor', 'grip0', 1, 0, 0)->('floor', 'grip0', 2, 0, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 1, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 1, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 1) = fail
node: ('floor', 'grip0', 0, 0, 1)->('floor', 'grip0', 0, 0, 2

...

node: ('grip0', 'goal', 0, 0, 0)->('grip0', 'goal', 1, 0, 0) = fail
node: ('floor', 'grip0', 2, 0, 0)->('floor', 'grip0', 2, 1, 0) = fail
node: ('floor', 'grip0', 0, 1, 0)->('floor', 'grip0', 0, 2, 0) = fail
branching: 946->1191 (52.36/300.0 s, steps/err: 39(78.8497924805 ms)/0.00185955022747)
node: ('goal', 'floor', 2, 0, 0)->('grip0', 'floor', 2, 0, 0) = fail
node: ('floor', 'grip0', 0, 0, 1)->('floor', 'grip0', 0, 0, 2) = fail
node: ('goal', 'floor', 2, 0, 0)->('goal', 'grip0', 2, 0, 0) = fail
node: ('goal', 'floor', 0, 2, 2)->('goal', 'floor', 1, 2, 2) = fail
node: ('floor', 'goal', 0, 0, 0)->('floor', 'goal', 0, 1, 0) = fail
node: ('floor', 'floor', 0, 2, 2)->('floor', 'floor', 1, 2, 2) = fail
node: ('grip0', 'goal', 2, 0, 0)->('grip0', 'goal', 2, 0, 1) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'grip0', 0, 0, 0) = fail
node: ('goal', 'grip0', 2, 2, 2)->('goal', 'goal', 2, 2, 2) = fail
node: ('floor', 'grip0', 0, 0, 1)->('floor', 'grip0', 0, 0, 2) = fail
node: ('floor', '

node: ('floor', 'goal', 2, 0, 0)->('floor', 'goal', 2, 0, 1) = fail
node: ('floor', 'goal', 0, 0, 0)->('grip0', 'goal', 0, 0, 0) = fail
node: ('grip0', 'goal', 2, 0, 0)->('grip0', 'goal', 2, 1, 0) = fail
node: ('grip0', 'floor', 2, 2, 0)->('grip0', 'floor', 2, 2, 1) = fail
node: ('floor', 'grip0', 0, 1, 0)->('floor', 'grip0', 0, 2, 0) = fail
node: ('grip0', 'goal', 2, 0, 2)->('grip0', 'goal', 2, 1, 2) = fail
node: ('goal', 'grip0', 2, 2, 1)->('goal', 'grip0', 2, 2, 2) = fail
node: ('grip0', 'goal', 2, 0, 0)->('goal', 'goal', 2, 0, 0) = fail
node: ('floor', 'grip0', 0, 0, 0)->('floor', 'grip0', 1, 0, 0) = success
branching: 372->1206 (53.09/300.0 s, steps/err: 20(102.199077606 ms)/0.00107777579679)
node: ('grip0', 'goal', 0, 0, 2)->('grip0', 'goal', 1, 0, 2) = fail
node: ('goal', 'grip0', 0, 0, 2)->('goal', 'grip0', 0, 1, 2) = success
node: ('grip0', 'floor', 2, 0, 2)->('grip0', 'floor', 2, 1, 2) = fail
branching: 1023->1207 (53.11/300.0 s, steps/err: 36(589.417934418 ms)/0.001712227995

node: ('goal', 'grip0', 0, 0, 0)->('goal', 'grip0', 1, 0, 0) = success
branching: 833->1223 (53.7/300.0 s, steps/err: 29(106.116056442 ms)/0.00149176594747)
branching: 615->1222 (53.7/300.0 s, steps/err: 36(1618.46399307 ms)/0.00118593406419)
node: ('goal', 'floor', 0, 0, 0)->('goal', 'floor', 0, 0, 1) = fail
node: ('grip0', 'goal', 0, 0, 0)->('grip0', 'goal', 0, 1, 0) = fail
node: ('floor', 'goal', 0, 0, 0)->('grip0', 'goal', 0, 0, 0) = fail
node: ('grip0', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 0, 1) = fail
node: ('grip0', 'floor', 0, 2, 2)->('grip0', 'floor', 1, 2, 2) = fail
node: ('floor', 'floor', 2, 2, 2)->('grip0', 'floor', 2, 2, 2) = fail
node: ('grip0', 'floor', 2, 0, 0)->('goal', 'floor', 2, 0, 0) = fail
node: ('floor', 'grip0', 2, 0, 2)->('floor', 'floor', 2, 0, 2) = success
branching: 294->1224 (53.76/300.0 s, steps/err: 24(39.4039154053 ms)/0.00172519417941)
node: ('grip0', 'floor', 2, 0, 2)->('floor', 'floor', 2, 0, 2) = fail
node: ('goal', 'floor', 2, 2, 2)->('grip0', 

node: ('floor', 'grip0', 2, 2, 2)->('floor', 'goal', 2, 2, 2) = success
node: ('goal', 'grip0', 0, 0, 0)->('goal', 'floor', 0, 0, 0) = success
branching: 1229->1238 (54.17/300.0 s, steps/err: 30(233.57796669 ms)/0.00148433540915)
node: ('goal', 'grip0', 2, 0, 2)->('goal', 'grip0', 2, 1, 2) = success
branching: 639->1240 (54.2/300.0 s, steps/err: 3(106.958866119 ms)/0.00144704334602)
node: ('grip0', 'goal', 0, 0, 0)->('floor', 'goal', 0, 0, 0) = success
branching: 636->1239 (54.19/300.0 s, steps/err: 24(92.4830436707 ms)/0.00194930839197)
node: ('goal', 'grip0', 2, 0, 0)->('goal', 'floor', 2, 0, 0) = success
branching: 547->1241 (54.22/300.0 s, steps/err: 7(38.5870933533 ms)/0.00163353261043)
======================= terminated 14: first answer acquired ===============================
branching: 187->1242 (54.21/300.0 s, steps/err: 27(414.627075195 ms)/0.00199672191715)
node: ('goal', 'grip0', 0, 0, 2)->('goal', 'grip0', 1, 0, 2) = success
======================= terminated 11: first ans

Process Process-30:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "pkg/planning/pipeline.py", line 145, in __search_loop
    display=display, dt_vis=dt_vis, **kwargs)
  File "pkg/planning/pipeline.py", line 191, in test_connection
Process Process-29:
    self.mplan.plan_transition(from_state, to_state, redundancy_dict=redundancy_dict, **kwargs)
Traceback (most recent call last):
  File "pkg/planning/motion/interface.py", line 102, in plan_transition
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    redundancy_values=redundancy_values, **kwargs)
    self.run()
  File "pkg/planning/motion/moveit/moveit_planner.py", line 235, in plan_algorithm
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    group_name, tool.geometry.link

In [29]:
print(gtimer)

plan: 	54287.0 ms/1 = 54286.89 ms (54286.89/54286.89)



## play searched plan

In [30]:
ppline.play_schedule(snode_schedule[:2], period=0.01)
pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
time.sleep(0.5)
gscene.show_pose(crob.home_pose)

In [31]:
snode_schedule = ppline.add_return_motion(snode_schedule)

In [35]:
ppline.play_schedule(snode_schedule, period=0.03)

In [33]:
len(snode_schedule)

24

In [53]:
for i_s,  snode in enumerate(snode_schedule):
    print("{}: {}".format(i_s, snode.state.node))

0: ('floor', 'floor', 0, 0, 0)
1: ('grip0', 'floor', 0, 0, 0)
2: ('goal', 'floor', 0, 0, 0)
3: ('grip0', 'floor', 0, 0, 0)
4: ('grip0', 'floor', 0, 0, 1)
5: ('grip0', 'floor', 0, 0, 2)
6: ('floor', 'floor', 0, 0, 2)
7: ('floor', 'grip0', 0, 0, 2)
8: ('floor', 'goal', 0, 0, 2)
9: ('grip0', 'goal', 0, 0, 2)
10: ('grip0', 'goal', 0, 1, 2)
11: ('floor', 'goal', 0, 1, 2)
12: ('grip0', 'goal', 0, 1, 2)
13: ('grip0', 'goal', 0, 2, 2)
14: ('grip0', 'goal', 1, 2, 2)
15: ('grip0', 'goal', 2, 2, 2)
16: ('floor', 'goal', 2, 2, 2)
17: ('floor', 'grip0', 2, 2, 2)
18: ('floor', 'floor', 2, 2, 2)
19: ('grip0', 'floor', 2, 2, 2)
20: ('goal', 'floor', 2, 2, 2)
21: ('goal', 'floor', 2, 2, 2)


## NOTE
* 위에 가운데 열 어떻게  0,1,1,1,1,2가 나오지? -> 일방통행 작업 플래그 추가
* goal-directed extension 추가해보기?
* 균등 샘플링 - 샘플 할때마다 노드별/전환별 확률 조정

In [56]:
tplan.node_dict[('grip0', 'goal', 0, 1, 2)]

{('floor', 'goal', 0, 1, 2),
 ('goal', 'goal', 0, 1, 2),
 ('grip0', 'goal', 0, 2, 2),
 ('grip0', 'goal', 1, 1, 2)}